In [31]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from func.prepro import SingleCellDataset

import gc
import joblib
import func.data as dt
import h5py
import hdf5plugin

import sklearn as sk
import pandas as pd
import numpy as np
import pathlib as pl

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
# from scipy.stats import pearsonr
# from sklearn.metrics import mean_squared_error as mse

from sklearn import datasets
from sklearn.linear_model import Lasso


In [5]:
RAW_DATA_DIR = pl.Path("D:/Data/open-problems-multimodal/raw")
PRO_DATA_DIR = pl.Path("D:/Data/open-problems-multimodal/processed")
METHOD_DIR = pl.Path("D:/OneDrive/OneDrive - UW-Madison/Kris/Code/SingleCell-Integration/methods")

DATA_NAME = "multi"
METHOD_NAME = "lasso"
BATCH_SIZE = 1000
BATCH_SIZE_PCA = 10000

FP_CELL_METADATA = pl.Path(RAW_DATA_DIR,"metadata.csv")
FP_SUBMISSION = pl.Path(RAW_DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = pl.Path(RAW_DATA_DIR,"evaluation_ids.csv")

FP_CITE_TRAIN_INPUTS = pl.Path(RAW_DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = pl.Path(RAW_DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = pl.Path(RAW_DATA_DIR,"test_cite_inputs.h5")

FP_MULTIOME_TRAIN_INPUTS = pl.Path(RAW_DATA_DIR,"train_multi_inputs.h5")
FP_MULTIOME_TRAIN_TARGETS = pl.Path(RAW_DATA_DIR,"train_multi_targets.h5")
FP_MULTIOME_TEST_INPUTS = pl.Path(RAW_DATA_DIR,"test_multi_inputs.h5")


In [52]:
test_multi = dt.readH5pyFile(RAW_DATA_DIR / f"test_{DATA_NAME}_inputs.h5")

MemoryError: Unable to allocate 47.7 GiB for an array with shape (55935, 228942) and data type float32

In [53]:
train_multi = SingleCellDataset(FP_MULTIOME_TRAIN_INPUTS,FP_MULTIOME_TRAIN_TARGETS)
test_multi = SingleCellDataset(FP_MULTIOME_TEST_INPUTS,FP_MULTIOME_TEST_INPUTS)

In [51]:
n_target = train_multi[0:BATCH_SIZE][2].shape[1]

In [61]:
print(n_target)

23418


In [25]:
# return a pca model and a standard scaler by training it with a large minibatch

def pre_pca(train_input, n_components, save=False): 
    train_sdsc = StandardScaler()
    train_input_sdsc = train_sdsc.fit_transform(train_input)
    train_pca = PCA(n_components=n_components)
    train_pca.fit(train_input_sdsc)

    if save == True:
        joblib.dump(train_sdsc, METHOD_DIR / f"train_{DATA_NAME}_sdsc.m")
        joblib.dump(train_pca, METHOD_DIR / f"train_{DATA_NAME}_pca.m")

    return train_pca, train_sdsc

In [62]:
# obtain a pca model by training it with a large minibatch

p = 500
train_pca, train_sdsc = pre_pca(train_multi[0:BATCH_SIZE_PCA][1], p, True)

In [60]:
y_pred = np.array([])
n_round = 1
train_loader_multi = DataLoader(train_multi, batch_size=BATCH_SIZE, num_workers=0, shuffle=False)
test_loader_multi = DataLoader(test_multi, batch_size=BATCH_SIZE, num_workers=0, shuffle=False)
for iy in range(n_target):
    print(iy)
    model = SGDRegressor(loss="squared_loss", penalty="l1", alpha=0.01)
    for round in range(n_round):
        ib = 0
        for cells, features, targets in train_loader_multi:
            print(ib)
            ib += 1
            model.partial_fit(features, targets[:,iy])
    
    y_pred_col = np.array([])
    for cells, features, features2 in test_loader_multi:
        y_pred_col = np.concatenate((y_pred_col, model.predict(features)), axis=0)

    y_pred = np.concatenate((y_pred, y_pred_col), axis=1)

np.savetxt(PRO_DATA_DIR / f"test_{DATA_NAME}_{METHOD_NAME}_pred.csv", y_pred, delimiter=",")
    


0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


KeyboardInterrupt: 

In [63]:
i = 0
train_loader_test = DataLoader(train_multi, batch_size=100, num_workers=0, shuffle=False)
for cells,features,targets in train_loader_test:
    print(i, features.shape, targets.shape)
    i += 1

0 torch.Size([100, 228942]) torch.Size([100, 23418])
1 torch.Size([100, 228942]) torch.Size([100, 23418])
2 torch.Size([100, 228942]) torch.Size([100, 23418])
3 torch.Size([100, 228942]) torch.Size([100, 23418])
4 torch.Size([100, 228942]) torch.Size([100, 23418])
5 torch.Size([100, 228942]) torch.Size([100, 23418])
6 torch.Size([100, 228942]) torch.Size([100, 23418])
7 torch.Size([100, 228942]) torch.Size([100, 23418])
8 torch.Size([100, 228942]) torch.Size([100, 23418])
9 torch.Size([100, 228942]) torch.Size([100, 23418])
10 torch.Size([100, 228942]) torch.Size([100, 23418])
11 torch.Size([100, 228942]) torch.Size([100, 23418])
12 torch.Size([100, 228942]) torch.Size([100, 23418])
13 torch.Size([100, 228942]) torch.Size([100, 23418])
14 torch.Size([100, 228942]) torch.Size([100, 23418])
15 torch.Size([100, 228942]) torch.Size([100, 23418])
16 torch.Size([100, 228942]) torch.Size([100, 23418])
17 torch.Size([100, 228942]) torch.Size([100, 23418])
18 torch.Size([100, 228942]) torch.Siz

In [33]:
class NaiveReg():
    def __init__(self, train_loader, pca, sdsc) -> None:
        self.pca = pca
        self.sdsc = sdsc
        self.train_loader = train_loader
        self.model = SGDRegressor(loss="squared_loss", penalty="l1", alpha=0.01)
        pass

    def fit(self, n_round=1):
        for i in range(n_round):
            for cells, features, targets in self.train_loader:
                self.partial_fit(features, targets)

    def partial_fit(self, feature, target):
        
        self.model.partial_fit(feature, target)

    def predict(self, test_input):
        self.model.predict

In [34]:
model = NaiveReg(train_loader_multi, train_pca, train_sdsc)
model.fit()

ValueError: y should be a 1d array, got an array of shape (1000, 140) instead.